<a href="https://colab.research.google.com/github/Sha-98/DSA-in-Python/blob/main/hangman_api_user.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved.
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server.

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

01. Check with vowels in the word,

**A. If vowels are found in the word:**
> Check for words with similar length, and similar place position of vowels in the dictionary.

> Count the letters appearance in these words. Guess with the letter with maximum number of appearence in the words filtered from the dictionary.

> Keep guessing with the letters found in the words filtered from the dictionary in the descending order of the number of times they are appearing in the words, altogether.

> If we get one more letter guessed correctly, check for the words with similar length and letter position in the dictionary again. Now, guess the letter which is appearing maximum times in these words.

> Keep rechecking and re-calculating the letter appearence in the filtered words until we get to one word filtered from the dictionary or we make the right guess and win the game. This word might be our correct guess. If there is only one word matching our conditions, guess with the letters present in these words, the letters must be the ones which are not guessed earlier.

**B. If vowels are not found in the word:**
> If no vowels are found in the word, check for the words in the dictionary with similar length and no vowels in them. Calculate the number of times each unique letter appears in these words altogether.

> Now, guess with the letter appearing most of the times. If we get the right guess process with filtering the words with similar length, no vowels, and similar position of the letter we just guessed.

> Now, re-arrange the letters found in the filtered words from the dictionary and count the number of times they appear.

> Now, guess with the letter appearing maximum times in these words until the right guess.

> Keep re-filtering the words from the dictionary and re-arranging the letters in these words in correct order to count the number of times they appear until we get only one word filtered or we get the correct guess in the game as this can be the correct answer for us.

01. We have a word with all letters hidden for us to guess them correctly

02. We start by guessing the vowels, one by one i.e. we check for 'a'. If 'a' is not found, we go on to guess with 'e' , 'i' , and so on until we make the first correct guess.

> **Situation A: We made a correct guess with one of the vowels**

> Now, as we know that we have the vowels in our words, and we also know which vowel it is along with the length of the word. We will take this information to filter the words with similar length, with same vowels present in them at the same position.

> We will now get a list of words with similar length, and vowels at the same position. Now, take the unique characters from these words, and count the number of times they have appeared in these words altogether. We will now make our next guess from among these letters.

> At this point we have a filtered list of letters and number of times they have apprared in the probably correct words filtered from the dictionary. Now, we will guess with the words in the descending order of their appearence one by one until we get our next correct guess.

> Now, at this moment, we have atleast 2 letters correctly guessed in our word. Now, we will go to the list of words filtered earlier, and now we will re-filter this list to get the words with similar length and position of letters guessed by us.

> Now, we have a shortened list, most probably. We will again count the number of times each unique letter is appeared in these words altogether and make a guess accordingly until we get the next correct guess. We will repeat step 4 and 5 of this section until we guess the correct word and win the game.

> **Situation B: There are no vowels in the word to be guessed**

> Now, we know that the word has no vowels and we also know the length of the word. Using this information, we will filter the words with similar length and no vowels from the dictionary. Now, we will calculate the number of times each unique letter is appearing in these words altogether.

> Now, we will guess with the letter in the descending order of their appearence until we get a correct guess once.

> At this point, we have atlest one letter guessed correctly in our word. Now, using the info of length of the letter, no vowels present, and the position of the correctly guessed letter, we will further filter the previous list of words to get an even shorter list of words. We will not re-arrange the unique letters present in these words, and calculate their appearence in the words altogether.

> We will now make our next guess with the letters in the descending order of their appearence in the latest filtered list of words, until we get our next correct guess.

> We will repeat the steps 3 and 4 until we finally make the correct guess and win the game.


In [1]:
!pip install urlparse

ERROR: Could not find a version that satisfies the requirement urlparse (from versions: none)
ERROR: No matching distribution found for urlparse


In [20]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [21]:
import collections
import requests
import time
from urllib.parse import parse_qs
from requests.exceptions import SSLError

class HangmanGuesser:
    def __init__(self, dictionary):
        self.dictionary = dictionary
        self.guessed_letters = set()

    def guess_with_vowels(self, clean_word):
        # Situation A: Vowels are present
        # Step 3.1: Filter words with similar length and same vowels
        filtered_words = [word for word in self.dictionary if len(word) == len(clean_word) and set(word) & set(clean_word)]

        # Step 3.2: Count the occurrence of each unique letter in the filtered words
        letter_count = collections.Counter("".join(filtered_words))

        # Step 3.3: Guess the letter with the maximum occurrence
        for letter, count in letter_count.most_common():
            if letter not in self.guessed_letters:
                self.guessed_letters.add(letter)
                return letter

    def guess_without_vowels(self, clean_word):
        # Situation B: No vowels are present
        # Step 4.1: Filter words with similar length and no vowels
        filtered_words = [word for word in self.dictionary if len(word) == len(clean_word) and not set(word) & set("AEIOU")]

        # Step 4.2: Count the occurrence of each unique letter in the filtered words
        letter_count = collections.Counter("".join(filtered_words))

        # Step 4.3: Guess the letter with the maximum occurrence
        for letter, count in letter_count.most_common():
            if letter not in self.guessed_letters:
                self.guessed_letters.add(letter)
                return letter

class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []

        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()

        self.current_dictionary = []

        # Create an instance of HangmanGuesser
        self.guesser = HangmanGuesser(self.full_dictionary)

# import collections
# import requests
# import time
# from urllib.parse import parse_qs
# from requests.exceptions import SSLError

# class HangmanGuesser:
#     def __init__(self, dictionary):
#         self.dictionary = dictionary
#         self.guessed_letters = set()

#     def guess_with_vowels(self, clean_word):
#         # Prioritize common consonants
#         common_consonants = "rstnlaeiou"

#         # Situation A: Vowels are present
#         # Step 3.1: Filter words with similar length and same vowels
#         filtered_words = [word for word in self.dictionary if len(word) == len(clean_word) and set(word) & set(clean_word)]

#         # Step 3.2: Count the occurrence of each unique letter in the filtered words
#         letter_count = collections.Counter("".join(filtered_words))

#         # Step 3.3: Guess the letter with the maximum occurrence
#         for letter in common_consonants:
#             if letter not in self.guessed_letters and letter in letter_count:
#                 self.guessed_letters.add(letter)
#                 return letter

#     def guess_without_vowels(self, clean_word):
#         # Prioritize common consonants
#         common_consonants = "rstnlaeiou"

#         # Situation B: No vowels are present
#         # Step 4.1: Filter words with similar length and no vowels
#         filtered_words = [word for word in self.dictionary if len(word) == len(clean_word) and not set(word) & set("AEIOU")]

#         # Step 4.2: Count the occurrence of each unique letter in the filtered words
#         letter_count = collections.Counter("".join(filtered_words))

#         # Step 4.3: Guess the letter with the maximum occurrence
#         for letter in common_consonants:
#             if letter not in self.guessed_letters and letter in letter_count:
#                 self.guessed_letters.add(letter)
#                 return letter

# class HangmanAPI(object):
#     def __init__(self, access_token=None, session=None, timeout=None):
#         self.hangman_url = self.determine_hangman_url()
#         self.access_token = access_token
#         self.session = session or requests.Session()
#         self.timeout = timeout
#         self.guessed_letters = []

#         full_dictionary_location = "words_250000_train.txt"
#         self.full_dictionary = self.build_dictionary(full_dictionary_location)
#         self.guesser = HangmanGuesser(self.full_dictionary)

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    # def guess(self, word): # word input example: "_ p p _ e "
    #     ###############################################
    #     # Replace with your own "guess" function here #
    #     ###############################################

    #     # clean the word so that we strip away the space characters
    #     # replace "_" with "." as "." indicates any character in regular expressions
    #     clean_word = word[::2].replace("_",".")

    #     # find length of passed word
    #     len_word = len(clean_word)

    #     # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
    #     current_dictionary = self.current_dictionary
    #     new_dictionary = []

    #     # iterate through all of the words in the old plausible dictionary
    #     for dict_word in current_dictionary:
    #         # continue if the word is not of the appropriate length
    #         if len(dict_word) != len_word:
    #             continue

    #         # if dictionary word is a possible match then add it to the current dictionary
    #         if re.match(clean_word,dict_word):
    #             new_dictionary.append(dict_word)

    #     # overwrite old possible words dictionary with updated version
    #     self.current_dictionary = new_dictionary


    #     # count occurrence of all characters in possible word matches
    #     full_dict_string = "".join(new_dictionary)

    #     c = collections.Counter(full_dict_string)
    #     sorted_letter_count = c.most_common()

    #     guess_letter = '!'

    #     # return most frequently occurring letter in all possible words that hasn't been guessed yet
    #     for letter,instance_count in sorted_letter_count:
    #         if letter not in self.guessed_letters:
    #             guess_letter = letter
    #             break

    #     # if no word matches in training dictionary, default back to ordering of full dictionary
    #     if guess_letter == '!':
    #         sorted_letter_count = self.full_dictionary_common_letter_sorted
    #         for letter,instance_count in sorted_letter_count:
    #             if letter not in self.guessed_letters:
    #                 guess_letter = letter
    #                 break

    #     return guess_letter

    def guess(self, word):
        clean_word = word[::2].replace("_", ".")

        if any(vowel in clean_word for vowel in "AEIOU"):
            return self.guesser.guess_with_vowels(clean_word)
        else:
            return self.guesser.guess_without_vowels(clean_word)

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################

    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)

                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))

                try:
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e

                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"

    def my_status(self):
        return self.request("/my_status", {})

    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result

class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

In [22]:
# class HangmanAPI(object):
#     def __init__(self, access_token=None, session=None, timeout=None):
#         self.hangman_url = self.determine_hangman_url()
#         self.access_token = access_token
#         self.session = session or requests.Session()
#         self.timeout = timeout
#         self.guessed_letters = []

#         full_dictionary_location = "words_250000_train.txt"
#         self.full_dictionary = self.build_dictionary(full_dictionary_location)
#         self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()

#         self.current_dictionary = []

#     @staticmethod
#     def determine_hangman_url():
#         links = ['https://trexsim.com', 'https://sg.trexsim.com']

#         data = {link: 0 for link in links}

#         for link in links:

#             requests.get(link)

#             for i in range(10):
#                 s = time.time()
#                 requests.get(link)
#                 data[link] = time.time() - s

#         link = sorted(data.items(), key=lambda x: x[1])[0][0]
#         link += '/trexsim/hangman'
#         return link

#     def guess(self, word): # word input example: "_ p p _ e "
#         ###############################################
#         # Replace with your own "guess" function here #
#         ###############################################

#         # clean the word so that we strip away the space characters
#         # replace "_" with "." as "." indicates any character in regular expressions
#         clean_word = word[::2].replace("_",".")

#         # find length of passed word
#         len_word = len(clean_word)

#         # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
#         current_dictionary = self.current_dictionary
#         new_dictionary = []

#         # iterate through all of the words in the old plausible dictionary
#         for dict_word in current_dictionary:
#             # continue if the word is not of the appropriate length
#             if len(dict_word) != len_word:
#                 continue

#             # if dictionary word is a possible match then add it to the current dictionary
#             if re.match(clean_word,dict_word):
#                 new_dictionary.append(dict_word)

#         # overwrite old possible words dictionary with updated version
#         self.current_dictionary = new_dictionary


#         # count occurrence of all characters in possible word matches
#         full_dict_string = "".join(new_dictionary)

#         c = collections.Counter(full_dict_string)
#         sorted_letter_count = c.most_common()

#         guess_letter = '!'

#         # return most frequently occurring letter in all possible words that hasn't been guessed yet
#         for letter,instance_count in sorted_letter_count:
#             if letter not in self.guessed_letters:
#                 guess_letter = letter
#                 break

#         # if no word matches in training dictionary, default back to ordering of full dictionary
#         if guess_letter == '!':
#             sorted_letter_count = self.full_dictionary_common_letter_sorted
#             for letter,instance_count in sorted_letter_count:
#                 if letter not in self.guessed_letters:
#                     guess_letter = letter
#                     break

#         return guess_letter



#     ##########################################################
#     # You'll likely not need to modify any of the code below #
#     ##########################################################

#     def build_dictionary(self, dictionary_file_location):
#         text_file = open(dictionary_file_location,"r")
#         full_dictionary = text_file.read().splitlines()
#         text_file.close()
#         return full_dictionary

#     def start_game(self, practice=True, verbose=True):
#         # reset guessed letters to empty set and current plausible dictionary to the full dictionary
#         self.guessed_letters = []
#         self.current_dictionary = self.full_dictionary

#         response = self.request("/new_game", {"practice":practice})
#         if response.get('status')=="approved":
#             game_id = response.get('game_id')
#             word = response.get('word')
#             tries_remains = response.get('tries_remains')
#             if verbose:
#                 print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
#             while tries_remains>0:
#                 # get guessed letter from user code
#                 guess_letter = self.guess(word)

#                 # append guessed letter to guessed letters field in hangman object
#                 self.guessed_letters.append(guess_letter)
#                 if verbose:
#                     print("Guessing letter: {0}".format(guess_letter))

#                 try:
#                     res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
#                 except HangmanAPIError:
#                     print('HangmanAPIError exception caught on request.')
#                     continue
#                 except Exception as e:
#                     print('Other exception caught on request.')
#                     raise e

#                 if verbose:
#                     print("Sever response: {0}".format(res))
#                 status = res.get('status')
#                 tries_remains = res.get('tries_remains')
#                 if status=="success":
#                     if verbose:
#                         print("Successfully finished game: {0}".format(game_id))
#                     return True
#                 elif status=="failed":
#                     reason = res.get('reason', '# of tries exceeded!')
#                     if verbose:
#                         print("Failed game: {0}. Because of: {1}".format(game_id, reason))
#                     return False
#                 elif status=="ongoing":
#                     word = res.get('word')
#         else:
#             if verbose:
#                 print("Failed to start a new game")
#         return status=="success"

#     def my_status(self):
#         return self.request("/my_status", {})

#     def request(
#             self, path, args=None, post_args=None, method=None):
#         if args is None:
#             args = dict()
#         if post_args is not None:
#             method = "POST"

#         # Add `access_token` to post_args or args if it has not already been
#         # included.
#         if self.access_token:
#             # If post_args exists, we assume that args either does not exists
#             # or it does not need `access_token`.
#             if post_args and "access_token" not in post_args:
#                 post_args["access_token"] = self.access_token
#             elif "access_token" not in args:
#                 args["access_token"] = self.access_token

#         time.sleep(0.2)

#         num_retry, time_sleep = 50, 2
#         for it in range(num_retry):
#             try:
#                 response = self.session.request(
#                     method or "GET",
#                     self.hangman_url + path,
#                     timeout=self.timeout,
#                     params=args,
#                     data=post_args,
#                     verify=False
#                 )
#                 break
#             except requests.HTTPError as e:
#                 response = json.loads(e.read())
#                 raise HangmanAPIError(response)
#             except requests.exceptions.SSLError as e:
#                 if it + 1 == num_retry:
#                     raise
#                 time.sleep(time_sleep)

#         headers = response.headers
#         if 'json' in headers['content-type']:
#             result = response.json()
#         elif "access_token" in parse_qs(response.text):
#             query_str = parse_qs(response.text)
#             if "access_token" in query_str:
#                 result = {"access_token": query_str["access_token"][0]}
#                 if "expires" in query_str:
#                     result["expires"] = query_str["expires"][0]
#             else:
#                 raise HangmanAPIError(response.json())
#         else:
#             raise HangmanAPIError('Maintype was not text, or querystring')

#         if result and isinstance(result, dict) and result.get("error"):
#             raise HangmanAPIError(result)
#         return result

# class HangmanAPIError(Exception):
#     def __init__(self, result):
#         self.result = result
#         self.code = None
#         try:
#             self.type = result["error_code"]
#         except (KeyError, TypeError):
#             self.type = ""

#         try:
#             self.message = result["error_description"]
#         except (KeyError, TypeError):
#             try:
#                 self.message = result["error"]["message"]
#                 self.code = result["error"].get("code")
#                 if not self.type:
#                     self.type = result["error"].get("type", "")
#             except (KeyError, TypeError):
#                 try:
#                     self.message = result["error_msg"]
#                 except (KeyError, TypeError):
#                     self.message = result

#         Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object.
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [18]:
api = HangmanAPI(access_token="652942a24acd461e57ed57a768b4cc", timeout=2000)

## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [29]:
api.start_game(practice=1,verbose=True)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


Successfully start a new game! Game ID: 9edb59836f89. # of tries remaining: 6. Word: _ _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': '9edb59836f89', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ '}
Guessing letter: a
Sever response: {'game_id': '9edb59836f89', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ a _ '}
Guessing letter: n
Sever response: {'game_id': '9edb59836f89', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ _ _ a _ '}
Guessing letter: r
Sever response: {'game_id': '9edb59836f89', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ _ _ _ a _ '}
Guessing letter: l
Sever response: {'game_id': '9edb59836f89', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ _ l _ a _ '}
Guessing letter: i
Sever response: {'game_id': '9edb59836f89', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ _ l i a _ '}
Guessing letter: u
Sever response: {'game_id': '9edb59836f89', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ u 

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [ ]:
# for i in range(1000):
#     print('Playing ', i, ' th game')
#     # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
#     #api.start_game(practice=0,verbose=False)

#     # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
#     time.sleep(0.5)

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [ ]:
# [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
# success_rate = total_recorded_successes/total_recorded_runs
# print('overall success rate = %.3f' % success_rate)